In [15]:
using JuMP
using HiGHS
using Plots
using Gurobi
using JuMP
using Statistics
const gurobi_env = Gurobi.Env() ; 
using DataFrames, CSV

Set parameter Username
Academic license - for non-commercial use only - expires 2025-10-28


In [16]:
## LOAD INPUTS
function load_input_generator(inputs_path)

	# Generators (and storage) data:
	generators = DataFrame(CSV.File(joinpath(inputs_path, "Generators_data.csv")))
	# Read fuels data
	fuels = DataFrame(CSV.File(joinpath(inputs_path, "Fuels_data.csv")))

	# Many of the columns in the input data will be unused (this is input format for the GenX model)
	# Select the ones we want for this model
	generators = select(generators, :R_ID, :Resource, :Zone, :THERM, :VRE, 
        :FLEX, :Flexible_Demand_Energy_Eff, :Max_Flexible_Demand_Delay, :Max_Flexible_Demand_Advance, :STOR, :HYDRO,
		:Commit, :Existing_Cap_MW, :Existing_Cap_MWh, :Cap_Size, :New_Build, :Max_Cap_MW,
		:Inv_Cost_per_MWyr, :Fixed_OM_Cost_per_MWyr, :Inv_Cost_per_MWhyr, :Fixed_OM_Cost_per_MWhyr,
		:Var_OM_Cost_per_MWh, :Start_Cost_per_MW, :Start_Fuel_MMBTU_per_MW, :Heat_Rate_MMBTU_per_MWh, :Fuel,
		:Min_Power, :Ramp_Up_Percentage, :Ramp_Dn_Percentage, :Up_Time, :Down_Time,
		:Eff_Up, :Eff_Down)
	# Set of all generators
	G = generators.R_ID

	# Calculate generator (and storage) total variable costs, start-up costs, 
	# and associated CO2 per MWh and per start
	generators.Var_Cost = zeros(Float64, size(G, 1))
	generators.CO2_Rate = zeros(Float64, size(G, 1))
	generators.Start_Cost = zeros(Float64, size(G, 1))
	generators.CO2_Per_Start = zeros(Float64, size(G, 1))
	for g in G
		# Variable cost ($/MWh) = variable O&M ($/MWh) + fuel cost ($/MMBtu) * heat rate (MMBtu/MWh)
		generators.Var_Cost[g] = generators.Var_OM_Cost_per_MWh[g] +
								 fuels[fuels.Fuel.==generators.Fuel[g], :Cost_per_MMBtu][1] * generators.Heat_Rate_MMBTU_per_MWh[g]
		# CO2 emissions rate (tCO2/MWh) = fuel CO2 content (tCO2/MMBtu) * heat rate (MMBtu/MWh)
		generators.CO2_Rate[g] = fuels[fuels.Fuel.==generators.Fuel[g], :CO2_content_tons_per_MMBtu][1] * generators.Heat_Rate_MMBTU_per_MWh[g]
		# Start-up cost ($/start/MW) = start up O&M cost ($/start/MW) + fuel cost ($/MMBtu) * start up fuel use (MMBtu/start/MW) 
		generators.Start_Cost[g] = generators.Start_Cost_per_MW[g] +
								   fuels[fuels.Fuel.==generators.Fuel[g], :Cost_per_MMBtu][1] * generators.Start_Fuel_MMBTU_per_MW[g]
		# Start-up CO2 emissions (tCO2/start/MW) = fuel CO2 content (tCO2/MMBtu) * start up fuel use (MMBtu/start/MW) 
		generators.CO2_Per_Start[g] = fuels[fuels.Fuel.==generators.Fuel[g], :CO2_content_tons_per_MMBtu][1] * generators.Start_Fuel_MMBTU_per_MW[g]
	end
	# Note: after this, we don't need the fuels Data Frame again...

	# Drop hydropower and biomass plants from generators set for simplicity 
	# (these are a small share of total ERCOT capacity, ~500 MW
	# G = intersect(generators.R_ID[.!(generators.HYDRO .== 1)], G)
	# G = intersect(generators.R_ID[.!(generators.NDISP .== 1)], G)

	generators[generators.STOR.>=1, :Inv_Cost_per_MWhyr]

	generators[generators.STOR.>=1, :Inv_Cost_per_MWyr]

	generators[generators.STOR.>=1, :Inv_Cost_per_MWhyr] .= 400

	return (generators, G)
end

load_input_generator (generic function with 1 method)

In [17]:
load_input_generator("/Users/laurah/Desktop/PRINCETON/junior fall/MAE573/power-systems-optimization/Project/WECC_6zone_2035_Cali")

(75×37 DataFrame
 Row │ R_ID   Resource                           Zone   THERM  VRE    FLEX   F ⋯
     │ Int64  String                             Int64  Int64  Int64  Int64  I ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │     1  CA_N_batteries_1                       1      0      0      1    ⋯
   2 │     2  CA_N_biomass_1                         1      0      0      0
   3 │     3  CA_N_conventional_hydroelectric_1      1      0      0      0
   4 │     4  CA_N_geothermal_1                      1      0      0      0
   5 │     5  CA_N_hydroelectric_pumped_storag…      1      0      0      1    ⋯
   6 │     6  CA_N_natural_gas_fired_combined_…      1      1      0      0
   7 │     7  CA_N_natural_gas_fired_combustio…      1      1      0      1
   8 │     8  CA_N_natural_gas_steam_turbine_1       1      1      0      0
   9 │     9  CA_N_onshore_wind_turbine_1            1      0      1      0    ⋯
  10 │    10  CA_N_other_peaker_1        

In [18]:
function load_input_demand(inputs_path)
	# Read demand input data and record parameters
	demand_inputs = DataFrame(CSV.File(joinpath(inputs_path, "Load_data.csv")))
	# Value of lost load (cost of involuntary non-served energy)
	VOLL = demand_inputs.Voll[1]
	# Set of price responsive demand (non-served energy) segments
	S = convert(Array{Int64}, collect(skipmissing(demand_inputs.Demand_Segment)))
	#NOTE:  collect(skipmising(input)) is needed here in several spots because the demand inputs are not 'square' (different column lengths)

	# Data frame for price responsive demand segments (nse)
	# NSE_Cost = opportunity cost per MWh of demand curtailment
	# NSE_Max = maximum % of demand that can be curtailed in each hour
	# Note that nse segment 1 = involuntary non-served energy (load shedding) at $9000/MWh
	# and segment 2 = one segment of voluntary price responsive demand at $600/MWh (up to 7.5% of demand)
	nse = DataFrame(Segment = S,
		NSE_Cost = VOLL .* collect(skipmissing(demand_inputs.Cost_of_Demand_Curtailment_per_MW)),
		NSE_Max = collect(skipmissing(demand_inputs.Max_Demand_Curtailment)))

	# Set of sequential hours per sub-period
	hours_per_period = convert(Int64, demand_inputs.Timesteps_per_Rep_Period[1])
	# Set of time sample sub-periods (e.g. sample days or weeks)
	P = convert(Array{Int64}, 1:demand_inputs.Rep_Periods[1])
	# Sub period cluster weights = number of periods (days/weeks) represented by each sample period
	W = convert(Array{Int64}, collect(skipmissing(demand_inputs.Sub_Weights)))
	# Set of all time steps
	T = convert(Array{Int64}, demand_inputs.Time_Index)
	# Create vector of sample weights, representing how many hours in the year
	# each hour in each sample period represents
	sample_weight = zeros(Float64, size(T, 1))
	t = 1
	for p in P
		for h in 1:hours_per_period
			sample_weight[t] = W[p] / hours_per_period
			t = t + 1
		end
	end
	# Set of zones 
	Z = convert(Array{Int64}, 1:2)

	# Load/demand time series by zone (TxZ array)
	demand = select(demand_inputs, :Load_MW_z1, :Load_MW_z2)
	# Uncomment this line to explore the data if you wish:
	# show(demand, allrows=true, allcols=true)

	return (demand, nse, sample_weight, hours_per_period, P, S, W, T, Z)
end

load_input_demand (generic function with 1 method)

In [19]:
load_input_demand("/Users/laurah/Desktop/PRINCETON/junior fall/MAE573/power-systems-optimization/Project/WECC_6zone_2035_Cali")

(2688×2 DataFrame
  Row │ Load_MW_z1  Load_MW_z2 
      │ Int64       Int64      
──────┼────────────────────────
    1 │      10590       16467
    2 │      11304       17591
    3 │      11218       17454
    4 │      11196       17422
    5 │      11168       17381
    6 │      11567       18000
    7 │      12158       18934
    8 │      11987       18654
    9 │      12354       19221
   10 │      12992       20215
   11 │      13683       21296
  ⋮   │     ⋮           ⋮
 2679 │      25352       39448
 2680 │      25584       39812
 2681 │      24819       38624
 2682 │      23105       35961
 2683 │      21721       33800
 2684 │      20760       32300
 2685 │      19966       31059
 2686 │      18606       28940
 2687 │      16833       26178
 2688 │      15160       23573
              2667 rows omitted, 4×3 DataFrame
 Row │ Segment  NSE_Cost  NSE_Max 
     │ Int64    Float64   Float64 
─────┼────────────────────────────
   1 │       1    2000.0    1.0
   2 │       2    1800.0 

In [20]:
function load_input_flex_demand(inputs_path)
    flex_demand_inputs = DataFrame(CSV.File(joinpath(inputs_path, "Flexible_load_availability.csv")))
    select!(flex_demand_inputs, Not(:Column1))
    return flex_demand_inputs
end

load_input_flex_demand (generic function with 1 method)

In [21]:
# load_input_flex_demand("/Users/laurah/Desktop/PRINCETON/junior fall/MAE573/power-systems-optimization/Project/WECC_6zone_2035_Cali")

In [22]:
function load_input_variability(inputs_path)
	# Read generator capacity factors by hour (used for variable renewables)
	# There is one column here for each resource (row) in the generators DataFrame
	variability = DataFrame(CSV.File(joinpath(inputs_path, "Generators_variability.csv")))
	# Drop the first column with row indexes, as these are unnecessary
	variability = variability[:, 2:ncol(variability)]

	return variability
end

load_input_variability (generic function with 1 method)

In [23]:
# load_input_variability("/Users/laurah/Desktop/PRINCETON/junior fall/MAE573/power-systems-optimization/Project/WECC_6zone_2035_Cali")

In [24]:
function load_input_network(inputs_path = "/Users/laurah/Desktop/PRINCETON/junior fall/MAE573/power-systems-optimization/Project/WECC_6zone_2035_Cali/")

	# Read network data
	network = DataFrame(CSV.File(joinpath(inputs_path, "Network.csv")))
	#Again, there is a lot of entries in here we will not use (formatted for GenX inputs), so let's select what we want
	# Array of network zones (z1, z2, z3)
	zones = collect(skipmissing(network.Network_Lines))
	# Network map showing lines connecting zones
	lines = select(network[1:1, :],
		:Network_Lines, :z1, :z2,
		:Line_Max_Flow_MW, :Line_Min_Flow_MW, :Line_Loss_Percentage,
		:Line_Max_Reinforcement_MW, :Line_Reinforcement_Cost_per_MWyr)
	# Add fixed O&M costs for lines = 1/20 of reinforcement cost
	lines.Line_Fixed_Cost_per_MW_yr = lines.Line_Reinforcement_Cost_per_MWyr ./ 20
	# Set of all lines
	L = convert(Array{Int64}, lines.Network_Lines)

	return (lines, L)
end

load_input_network (generic function with 2 methods)

In [25]:
function hoursbefore(p::Int, t::Int, b::UnitRange{Int})::Vector{Int}
    period = div(t - 1, p)
    return period * p .+ mod1.(t .- b, p)
end

function hoursbefore(p::Int, t::Int, b::Int)::Int
    period = div(t - 1, p)
    return period * p + mod1(t - b, p)
end

function hoursafter(p::Int, t::Int, a::UnitRange{Int})::Vector{Int}
    period = div(t - 1, p)
    return period * p .+ mod1.(t .+ a, p)
end

hoursafter (generic function with 1 method)

In [30]:
function solve_cap_expansion(generators, G,
	demand, S, T, Z,
	nse,
	sample_weight, hours_per_period,
	lines, L,
	variability, flex_percentage, solver)

	#SUBSETS
	# By naming convention, all subsets are UPPERCASE

	# Subset of G of all thermal resources subject to unit commitment constraints
	UC = intersect(generators.R_ID[generators.Commit.==1], G)
	# Subset of G NOT subject to unit commitment constraints
	ED = intersect(generators.R_ID[.!(generators.Commit .== 1)], G)
	# Subset of G of all storage resources
	STOR = intersect(generators.R_ID[generators.STOR.>=1], G)
	# Subset of G of all variable renewable resources
	VRE = intersect(generators.R_ID[generators.VRE.==1], G)
	# Subset of all new build resources
	NEW = intersect(generators.R_ID[generators.New_Build.==1], G)
	# Subset of all existing resources
	OLD = intersect(generators.R_ID[.!(generators.New_Build .== 1)], G)
    # Subset of all flexible resources
    FLEX = intersect(generators.R_ID[(generators.FLEX .== 1)], G)

	Expansion_Model = Model(solver)

	# DECISION VARIABLES
	# By naming convention, all decision variables start with v and then are in UPPER_SNAKE_CASE

	# Capacity decision variables
	@variables(Expansion_Model, begin
		vCAP[g in G] >= 0     # power capacity (MW)
		vRET_CAP[g in OLD] >= 0     # retirement of power capacity (MW)
		vNEW_CAP[g in NEW] >= 0     # new build power capacity (MW)

		vE_CAP[g in STOR] >= 0     # storage energy capacity (MWh)
		vRET_E_CAP[g in intersect(STOR, OLD)] >= 0     # retirement of storage energy capacity (MWh)
		vNEW_E_CAP[g in intersect(STOR, NEW)] >= 0     # new build storage energy capacity (MWh)

		vT_CAP[l in L] >= 0     # transmission capacity (MW)
		vRET_T_CAP[l in L] >= 0     # retirement of transmission capacity (MW)
		vNEW_T_CAP[l in L] >= 0     # new build transmission capacity (MW)
	end)

    # !!! FLEX DEMAND !!!

    # # Variable tracking total advanced (negative) or deferred (positive) demand for demand flex resource y in period t
    # @variable(Expansion_Model, vS_FLEX[t in T, g in FLEX])
    # # Variable tracking demand deferred by demand flex resource y in period t
    # @variable(Expansion_Model, vCHARGE_FLEX[t in T, g in FLEX]>=0)
    # @variable(Expansion_Model, vF_CAP[t in T, g in FLEX] >= 0) # "storage" energy FLEX EV load (MWh)

	# Set upper bounds on capacity for renewable resources 
	# (which are limited in each resource 'cluster')
	for g in NEW[generators[NEW, :Max_Cap_MW].>0]
		set_upper_bound(vNEW_CAP[g], generators.Max_Cap_MW[g])
	end

	# Set upper bounds on transmission capacity expansion
	for l in L
		set_upper_bound(vNEW_T_CAP[l], lines.Line_Max_Reinforcement_MW[l])
	end

	# Operational decision variables
	@variables(Expansion_Model, begin
		vGEN[T, G] >= 0  # Power generation (MW)
		vCHARGE[T, STOR] >= 0  # Power charging (MW)
		vSOC[T, STOR] >= 0  # Energy storage state of charge (MWh)
		vNSE[T, S, Z] >= 0  # Non-served energy/demand curtailment (MW)
		vFLOW[T, L]      # Transmission line flow (MW); 
		# note line flow is positive if flowing
		# from source node (indicated by 1 in zone column for that line) 
		# to sink node (indicated by -1 in zone column for that line); 
		# flow is negative if flowing from sink to source.
	end)

	# CONSTRAINTS
	# By naming convention, all constraints start with c and then are TitleCase

    # (0) Mass-based CO2 Constraint for California
    @expression(Expansion_Model, eEmissionsByZone[z in Z],
        sum(generators.CO2_Rate[g] * vGEN[t, g] * sample_weight[t] * (generators.Zone[g] == z) for t in T, g in G) + 
        sum(generators.CO2_Per_Start[g] * (generators.Zone[g] == z) for g in G)
    )
    @constraint(Expansion_Model, cCO2Emissions_systemwide,
        sum(eEmissionsByZone[z] for z in Z) <= 25e6  # Absolute emissions limit in tons
    )
    
    # !!! FLEX DEMAND !!!
        # Power balance expressions
    # @expression(Expansion_Model, ePowerBalanceDemandFlex[t in T, z in Z],
    # sum(vGEN[t, g] + vCHARGE_FLEX[t, g]
    # for g in intersect(generators[generators.Zone .== z, :R_ID], FLEX))
    # )
    
	# (1) Supply-demand balance constraint for all time steps and zones
	@constraint(Expansion_Model, cDemandBalance[t in T, z in Z],
		sum(vGEN[t, g] for g in intersect(generators[generators.Zone.==z, :R_ID], G)) +
		sum(vNSE[t, s, z] for s in S) -
		sum(vCHARGE[t, g] for g in intersect(generators[generators.Zone.==z, :R_ID], STOR)) -
		demand[t, z] -
		sum(lines[l, Symbol(string("z", z))] * vFLOW[t, l] for l in L) == 0
	)

	# (2-6) Capacitated constraints:
	@constraints(Expansion_Model, begin
		# (2) Max power constraints for all time steps and all generators/storage
		cMaxPower[t in T, g in G], vGEN[t, g] <= variability[t, g] * vCAP[g]
		# (3) Max charge constraints for all time steps and all storage resources
		cMaxCharge[t in T, g in STOR], vCHARGE[t, g] <= vCAP[g]
		# (4) Max state of charge constraints for all time steps and all storage resources
		cMaxSOC[t in T, g in STOR], vSOC[t, g] <= vE_CAP[g]
		# (5) Max non-served energy constraints for all time steps and all segments and all zones
		cMaxNSE[t in T, s in S, z in Z], vNSE[t, s, z] <= nse.NSE_Max[s] * demand[t, z]
		# (6a) Max flow constraints for all time steps and all lines
		cMaxFlow[t in T, l in L], vFLOW[t, l] <= vT_CAP[l]
		# (6b) Min flow constraints for all time steps and all lines
		cMinFlow[t in T, l in L], vFLOW[t, l] >= -vT_CAP[l]
	end)

	# (7-9) Total capacity constraints:
	@constraints(Expansion_Model, begin
		# (7a) Total capacity for existing units
		cCapOld[g in OLD], vCAP[g] == generators.Existing_Cap_MW[g] - vRET_CAP[g]
		# (7b) Total capacity for new units
		cCapNew[g in NEW], vCAP[g] == vNEW_CAP[g]

		# (8a) Total energy storage capacity for existing units
		cCapEnergyOld[g in intersect(STOR, OLD)],
		vE_CAP[g] == generators.Existing_Cap_MWh[g] - vRET_E_CAP[g]
		# (8b) Total energy storage capacity for existing units
		cCapEnergyNew[g in intersect(STOR, NEW)],
		vE_CAP[g] == vNEW_E_CAP[g]

		# (9) Total transmission capacity
		cTransCap[l in L], vT_CAP[l] == lines.Line_Max_Flow_MW[l] - vRET_T_CAP[l] + vNEW_T_CAP[l]
	end)

    # (10) !!! FLEX DEMAND !!!
        # Capacity constraint on max shiftable demand at a given time based on "Flexible_load_availability"
    # for z in Z
    #     if z == 1
    #         col_name = "ev_load_shifting_z1"
    #     elseif z == 2
    #         col_name = "ev_load_shifting_z2"
    #     else
    #         error("Zone $z does not have a valid column name mapping!")
    #     end

    #     FLEX_Z = intersect(FLEX, generators[generators.Zone .== z, :R_ID])
    #     @constraint(Expansion_Model, [t in T, g in FLEX_Z],
    #         vF_CAP[t, g] <= flex_percentage[t, col_name] * demand[t, z]
    #     )
    # end

    # for z in Z
    #     FLEX_Z = intersect(FLEX, generators[generators.Zone .== z, :R_ID])
    
    #     # Define constraints for flexible demand in the zone
    #     @constraints(Expansion_Model,
    #         begin
    #             # State of charge balance constraint
    #             [t in T, g in FLEX_Z],
    #             vS_FLEX[t, g] == vS_FLEX[hoursbefore(hours_per_period, t, 1), g] -
    #                             generators.Flexible_Demand_Energy_Eff[g] * vGEN[t, g] +
    #                             vCHARGE_FLEX[t, g]
    
    #             # Maximum charging rate
    #             [t in T, g in FLEX_Z],
    #             vCHARGE_FLEX[t, g] <= vF_CAP[t, g]
    
    #             # Maximum discharging rate
    #             [t in T, g in FLEX_Z],
    #             generators.Flexible_Demand_Energy_Eff[g] * vGEN[t, g] <= vF_CAP[t, g]
    #         end
    #     )
    
    #     for g in FLEX_Z
    #         max_flex_demand_delay = 5  # MANUALLY SET
    #         max_flex_demand_advance = 0  # MANUALLY SET
    
    #         # Delay constraints
    #         if max_flex_demand_delay > 0
    #             @constraint(Expansion_Model, [t in T],
    #                 sum(vGEN[e, g]
    #                     for e in hoursafter(hours_per_period, t, 1:max_flex_demand_delay)) >= vS_FLEX[t, g]
    #             )
    #         end
    
    #         # Advance constraints
    #         if max_flex_demand_advance > 0
    #             @constraint(Expansion_Model, [t in T],
    #                 sum(vCHARGE_FLEX[e, g]
    #                     for e in hoursafter(hours_per_period, t, 1:max_flex_demand_advance)) >= -vS_FLEX[t, g]
    #             )
    #         end
    #     end
    # end
            
	# Because we are using time domain reduction via sample periods (days or weeks),
	# we must be careful with time coupling constraints at the start and end of each
	# sample period. 

	# First we record a subset of time steps that begin a sub period 
	# (these will be subject to 'wrapping' constraints that link the start/end of each period)
	# We include some additional logic for the case of 52 weeks because the full 8760 does not line up exactly
	# with 52 weeks. There is one extra day. This logic ignores the last "start".
	STARTS = convert(Array{Int64}, 1:hours_per_period:floor(maximum(T) / hours_per_period)*hours_per_period)
	# Then we record all time periods that do not begin a sub period 
	# (these will be subject to normal time couping constraints, looking back one period)
	INTERIORS = setdiff(T, STARTS)

	# (10-12) Time coupling constraints
	@constraints(Expansion_Model, begin
		# (10a) Ramp up constraints, normal
		cRampUp[t in INTERIORS, g in G],
		vGEN[t, g] - vGEN[t-1, g] <= generators.Ramp_Up_Percentage[g] * vCAP[g]
		# (10b) Ramp up constraints, sub-period wrapping
		cRampUpWrap[t in STARTS, g in G],
		vGEN[t, g] - vGEN[t+hours_per_period-1, g] <= generators.Ramp_Up_Percentage[g] * vCAP[g]

		# (11a) Ramp down, normal
		cRampDown[t in INTERIORS, g in G],
		vGEN[t-1, g] - vGEN[t, g] <= generators.Ramp_Dn_Percentage[g] * vCAP[g]
		# (11b) Ramp down, sub-period wrapping
		cRampDownWrap[t in STARTS, g in G],
		vGEN[t+hours_per_period-1, g] - vGEN[t, g] <= generators.Ramp_Dn_Percentage[g] * vCAP[g]

		# (12a) Storage state of charge, normal
		cSOC[t in INTERIORS, g in STOR],
		vSOC[t, g] == vSOC[t-1, g] + generators.Eff_Up[g] * vCHARGE[t, g] - vGEN[t, g] / generators.Eff_Down[g]
		# (12a) Storage state of charge, wrapping
		cSOCWrap[t in STARTS, g in STOR],
		vSOC[t, g] == vSOC[t+hours_per_period-1, g] + generators.Eff_Up[g] * vCHARGE[t, g] - vGEN[t, g] / generators.Eff_Down[g]
	end)

	# Create expressions for each sub-component of the total cost (for later retrieval)
        # !!! FLEX DEMAND !!!
    # @expression(Expansion_Model, eCVarFlex_in[t in T, g in FLEX],
    #     sample_weight[t] * generators.Var_OM_Cost_per_MWh[g] * vCHARGE_FLEX[t, g])
    # @expression(Expansion_Model, eTotalCVarFlexInT[t in T],
    #     sum(eCVarFlex_in[t, g] for g in FLEX)
    # )
    # @expression(Expansion_Model, eTotalCVarFlexIn, # CUMULATIVE FLEX VAR COST
    #     sum(eTotalCVarFlexInT[t] for t in T)
    # )
    
	@expression(Expansion_Model, eFixedCostsGeneration,
		# Fixed costs for total capacity 
		sum(generators.Fixed_OM_Cost_per_MWyr[g] * vCAP[g] for g in G) +
		# Investment cost for new capacity
		sum(generators.Inv_Cost_per_MWyr[g] * vNEW_CAP[g] for g in NEW)
	)
	@expression(Expansion_Model, eFixedCostsStorage,
		# Fixed costs for total storage energy capacity 
		sum(generators.Fixed_OM_Cost_per_MWhyr[g] * vE_CAP[g] for g in STOR) +
		# Investment costs for new storage energy capacity
		sum(generators.Inv_Cost_per_MWhyr[g] * vNEW_CAP[g] for g in intersect(STOR, NEW))
	)
	@expression(Expansion_Model, eFixedCostsTransmission,
		# Investment and fixed O&M costs for transmission lines
		sum(lines.Line_Fixed_Cost_per_MW_yr[l] * vT_CAP[l] +
			lines.Line_Reinforcement_Cost_per_MWyr[l] * vNEW_T_CAP[l] for l in L)
	)
	@expression(Expansion_Model, eVariableCosts,
		# Variable costs for generation, weighted by hourly sample weight
		sum(sample_weight[t] * generators.Var_Cost[g] * vGEN[t, g] for t in T, g in G)
	)
	@expression(Expansion_Model, eNSECosts,
		# Non-served energy costs
		sum(sample_weight[t] * nse.NSE_Cost[s] * vNSE[t, s, z] for t in T, s in S, z in Z)
	)

	@expression(Expansion_Model, eTotalCosts,
		eFixedCostsGeneration + eFixedCostsStorage + eFixedCostsTransmission +
		eVariableCosts + eNSECosts
	)

	@objective(Expansion_Model, Min,
		eTotalCosts
	)

	#     optimize!(Expansion_Model)
	time = @elapsed optimize!(Expansion_Model)

	# Record generation capacity and energy results
	generation = zeros(size(G, 1))
	for i in 1:size(G, 1)
		# Note that total annual generation is sumproduct of sample period weights and hourly sample period generation 
		generation[i] = sum(sample_weight .* value.(vGEN)[:, G[i]].data)
	end
	# Total annual demand is sumproduct of sample period weights and hourly sample period demands
	total_demand = sum(sum.(eachcol(sample_weight .* demand)))
	# Maximum aggregate demand is the maximum of the sum of total concurrent demand in each hour
	peak_demand = maximum(sum(eachcol(demand)))
	MWh_share = generation ./ total_demand .* 100
	cap_share = value.(vCAP).data ./ peak_demand .* 100
	generator_results = DataFrame(
		ID = G,
		Resource = generators.Resource[G],
		Zone = generators.Zone[G],
		Total_MW = value.(vCAP).data,
		Start_MW = generators.Existing_Cap_MW[G],
		Change_in_MW = value.(vCAP).data .- generators.Existing_Cap_MW[G],
		Percent_MW = cap_share,
		GWh = generation / 1000,
		Percent_GWh = MWh_share,
	)

	# Record energy storage energy capacity results (MWh)
	storage_results = DataFrame(
		ID = STOR,
		Zone = generators.Zone[STOR],
		Resource = generators.Resource[STOR],
		Total_Storage_MWh = value.(vE_CAP).data,
		Start_Storage_MWh = generators.Existing_Cap_MWh[STOR],
		Change_in_Storage_MWh = value.(vE_CAP).data .- generators.Existing_Cap_MWh[STOR],
	)


	# Record transmission capacity results
	transmission_results = DataFrame(
		Line = L,
		Total_Transfer_Capacity = value.(vT_CAP).data,
		Start_Transfer_Capacity = lines.Line_Max_Flow_MW,
		Change_in_Transfer_Capacity = value.(vT_CAP).data .- lines.Line_Max_Flow_MW,
	)


	## Record non-served energy results by segment and zone
	num_segments = maximum(S)
	num_zones = maximum(Z)
	nse_results = DataFrame(
		Segment = zeros(num_segments * num_zones),
		Zone = zeros(num_segments * num_zones),
		NSE_Price = zeros(num_segments * num_zones),
		Max_NSE_MW = zeros(num_segments * num_zones),
		Total_NSE_MWh = zeros(num_segments * num_zones),
		NSE_Percent_of_Demand = zeros(num_segments * num_zones),
	)
	i = 1
	for s in S
		for z in Z
			nse_results.Segment[i] = s
			nse_results.Zone[i] = z
			nse_results[!, :NSE_Price] .= nse.NSE_Cost[s]
			nse_results.Max_NSE_MW[i] = maximum(value.(vNSE)[:, s, z].data)
			nse_results.Total_NSE_MWh[i] = sum(sample_weight .* value.(vNSE)[:, s, z].data)
			nse_results.NSE_Percent_of_Demand[i] = sum(sample_weight .* value.(vNSE)[:, s, z].data) / total_demand * 100
			i = i + 1
		end
	end

	# Record costs by component (in million dollars)
	# Note: because each expression evaluates to a single value, 
	# value.(JuMPObject) returns a numerical value, not a DenseAxisArray;
	# We thus do not need to use the .data extension here to extract numeric values
	cost_results = DataFrame(
		Fixed_Costs_Generation = value.(eFixedCostsGeneration) / 10^6,
		Fixed_Costs_Storage = value.(eFixedCostsStorage) / 10^6,
		Fixed_Costs_Transmission = value.(eFixedCostsTransmission) / 10^6,
		Variable_Costs = value.(eVariableCosts) / 10^6,
		NSE_Costs = value.(eNSECosts) / 10^6,
		Total_Costs = value.(eTotalCosts) / 10^6,
	)

    flexible_demand_results = DataFrame(A = [0]) # placeholder
    # Record flexible demand results
    # flexible_demand_results = DataFrame(
    #     Time = repeat(T, inner=length(FLEX)),
    #     Resource = vcat([fill(string(g), length(T)) for g in FLEX]...),
    #     Total_Deferred_Advanced_Demand = vec(value.(vS_FLEX)),
    #     Demand_Deferred = vec(value.(vCHARGE_FLEX)),
    #     Storage_Capacity = vec(value.(vF_CAP)),
    # )

    # Record Average Cost per MWh
    # total_cost = value.(eTotalCosts) / 10^6
    # total_demand_served = sum(sample_weight[t] * sum(value.(vGEN)[t, g] for g in G) for t in T)
    # avg_cost_per_mwh = total_cost / total_demand_served


	return (generator_results,
		storage_results,
		transmission_results,
		nse_results,
		cost_results,
        flexible_demand_results,
		time)
end

solve_cap_expansion (generic function with 1 method)

In [27]:
function write_results(generator_results,
	storage_results,
	transmission_results,
	nse_results,
	cost_results,
    flex_results,
	time,
	outpath)

	# If output directory does not exist, create it
	if !(isdir(outpath))
		mkdir(outpath)
	end

	CSV.write(joinpath(outpath, "generator_results.csv"), generator_results)
	CSV.write(joinpath(outpath, "storage_results.csv"), storage_results)
	CSV.write(joinpath(outpath, "transmission_results.csv"), transmission_results)
	CSV.write(joinpath(outpath, "nse_results.csv"), nse_results)
	CSV.write(joinpath(outpath, "cost_results.csv"), cost_results)
    CSV.write(joinpath(outpath, "flex_results.csv"), flex_results)
	CSV.write(joinpath(outpath, "time_results.csv"), DataFrame(time = time))

end

write_results (generic function with 1 method)

In [32]:
inputs_path = "WECC_6zone_2035_Cali"

# REPLACE THIS WITH YOUR PATH TO WHERE YOU WANT TO OUTPUT DATA:
outputs_directory = "/Users/laurah/Desktop/PRINCETON/junior fall/MAE573/power-systems-optimization/Project/SCENARIO_1"
if !(isdir(outputs_directory))
	mkdir(outputs_directory)
end

### STANDARD MODEL ###########

# load inputs
(generators, G) = load_input_generator(inputs_path)
(demand, nse, sample_weight, hours_per_period, P, S, W, T, Z) = load_input_demand(inputs_path)
variability = load_input_variability(inputs_path)
(lines, L) = load_input_network(inputs_path);
flex_percentage = load_input_flex_demand(inputs_path);

In [33]:
# solve model
(generator_results,
	storage_results,
	transmission_results,
	nse_results,
	cost_results,
    flex_results,
	time_112_days) = solve_cap_expansion(generators, G,
	demand, S, T, Z,
	nse,
	sample_weight, hours_per_period,
	lines, L,
	variability, flex_percentage, Gurobi.Optimizer)

# report time
time_112_days = round(time_112_days, digits = 2)
"Time elapsed to solve with 112 days: $time_112_days seconds"

# write results
write_results(generator_results,
	storage_results,
	transmission_results,
	nse_results,
	cost_results,
    flex_results,
	time_112_days,
	joinpath(outputs_directory, "112_days"))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-10-28
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.0.0 24A335)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 685523 rows, 258213 columns and 2041153 nonzeros
Model fingerprint: 0xdc0f866b
Coefficient statistics:
  Matrix range     [1e-03, 9e+00]
  Objective range  [1e-01, 5e+05]
  Bounds range     [2e+02, 6e+04]
  RHS range        [3e+00, 2e+07]
Presolve removed 70275 rows and 12698 columns
Presolve time: 1.52s
Presolved: 232075 rows, 647570 columns, 2079671 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 2.27s

Barrier statistics:
 Dense cols : 1
 Free vars  : 8064
 AA' NZ     : 5.780e+06
 Factor NZ  : 3.036e+07 (roughly 600 MB of memory)
 Factor Ops : 4.669e+09 (less than 1 second per iteration)
 Threads    : 6

            

"/Users/laurah/Desktop/PRINCETON/junior fall/MAE573/power-systems-optimization/Project/SCENARIO_1/112_days/time_results.csv"